<a href="https://colab.research.google.com/github/HARSITHRAM/Interactive-Online-Class/blob/main/Copy_of_emotion_detection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!wget https://github.com/googlefonts/roboto/raw/main/src/hinted/Roboto-Bold.ttf -O /content/bold.ttf
# Download the Roboto-Bold.ttf font and save it as bold.ttf

from IPython import get_ipython
from IPython.display import display
# %%
!apt-get update
!apt-get install -y libfreetype6-dev libpng-dev
!pip install -U Pillow
!pip install deepface

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from deepface import DeepFace
from PIL import Image, ImageDraw, ImageFont


# Function to convert JavaScript object into OpenCV image
def js_to_image(js_reply):
    print("js_to_image called")  # Debugging print
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    print("Image shape:", img.shape)  # Debugging print
    return img


# Function to overlay text on an image
def text_to_bytes(bbox_array, text="Vision Programmer", text_position=(150, 275), font_size=50):
    print("text_to_bytes called")  # Debugging print
    if bbox_array.shape != (480, 640, 4):
        raise ValueError("bbox_array must have the shape (480, 640, 4)")

    img = Image.fromarray(bbox_array, 'RGBA')
    draw = ImageDraw.Draw(img)

    try:
        font_path = '/content/bold.ttf'  # Path where the font is downloaded - you might need to adjust this
        font = ImageFont.truetype(font_path, font_size)
    except IOError:
        raise IOError("Font file not found. Ensure the font is downloaded correctly.")

    draw.text(text_position, text, fill=(0, 0, 255, 255), font=font)

    iobuf = io.BytesIO()
    img.save(iobuf, format='png')

    img_bytes = 'data:image/png;base64,{}'.format(
        b64encode(iobuf.getvalue()).decode('utf-8')
    )

    print("Overlay image generated")  # Debugging print
    return img_bytes


# Function to display the video stream
def video_stream():
    js = Javascript('''
        async function stream_frame(label, bbox) {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const ctx = canvas.getContext('2d');

            // Get access to the user's camera
            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            await video.play();

            // Set canvas dimensions
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;

            // Draw video frame and overlay
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);

            // Convert canvas content to data URL
            const img = canvas.toDataURL('image/jpeg');

            // Return the data
            return { img };
        }
    ''')
    display(js)
    print("Video stream started")  # Debugging print


# Function to get a video frame
def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    print("Video frame received")  # Debugging print
    return data


# Main execution loop
video_stream()
label_html = 'Capturing...'
bbox = ''

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    img0 = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    try:
        print("Analyzing emotion...")  # Debugging print
        # Setting enforce_detection to False to avoid the ValueError if no face is detected
        objs = DeepFace.analyze(img_path=img0, actions=['emotion'], enforce_detection=False)
        emotion = objs[0]['dominant_emotion']
        print("Detected emotion:", emotion)  # Debugging print
        base64 = text_to_bytes(bbox_array, text=emotion)
        bbox = base64
    except Exception as e:
        print(f"Error during emotion analysis: {e}")
        base64 = text_to_bytes(bbox_array)
        bbox = base64

--2025-03-02 16:16:07--  https://github.com/googlefonts/roboto/raw/main/src/hinted/Roboto-Bold.ttf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/googlefonts/roboto-2/raw/main/src/hinted/Roboto-Bold.ttf [following]
--2025-03-02 16:16:07--  https://github.com/googlefonts/roboto-2/raw/main/src/hinted/Roboto-Bold.ttf
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/googlefonts/roboto-2/main/src/hinted/Roboto-Bold.ttf [following]
--2025-03-02 16:16:07--  https://raw.githubusercontent.com/googlefonts/roboto-2/main/src/hinted/Roboto-Bold.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.

<IPython.core.display.Javascript object>

Video stream started
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: sad
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: sad
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: neutral
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: sad
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: sad
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Image shape: (480, 640, 3)
Analyzing emotion...
Detected emotion: happy
text_to_bytes called
Overlay image generated
Video frame received
js_to_image called
Ima

KeyboardInterrupt: 